<a href="https://colab.research.google.com/github/lfernandof/causal_inference/blob/main/slippery_grounds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>

## Erro de português
>Quando o português chegou \\
>debaixo duma bruta chuva \\
>vestiu o índio \\
>que pena! \\
>fosse uma manhã de sol \\
>o índio tinha despido o português \\

</center>

<right>Oswald de Andrade, 1927</right>

\\

---

\\

Aquaplanagem, menor visibilidade, risco de derrapamento: condições climáticas advsersas podem impactar a maneira como pessoas dirigem e a segurança na via. Além do senso comum, o manual de direção defensiva recomenda uma maior distância de seguimento e menor velocidade em condições de tempo adversas. No célebre poema "Erro de Português" apresentado acima, Oswald de Andrade (1890-1954) entreteve: _"fosse uma manhã de sol..."_

\\

Com este estudo, pretendo uma resposta tentativa a esse questionamento: **Em um mundo contrafactual, onde os motoristas não estão sujeitos às intempéries, observaríamos a mesma taxa de fatalidade em acidentes nas estradas brasileiras?**

\\

Para responder a essa questão eu vou utilizar o *paradigma de desfechos potenciais* (potential outcomes), também conhecido por *modelo causal de Neyman-Rubin*, aplicados a dados de acidentes de trânsito em território nacional, coletados entre 2007–2021 e compilados pela Polícia Rodoviária Federal (PRF), gentilmente [disponibilizados como um data set público no Kaggle](https://www.kaggle.com/datasets/mcamera/brazil-highway-traffic-accidents/data).


\\

---

\\


# Interações esperadas

Nossas interação causa-efeito alvo será entre as seguintes variáveis: condição climática e um "score de fatalidade" (quociente entre o número de vítimas fatais em uma ocorrência, coluna _mortos_, e o número de passageiros envolvidos, coluna _pessoas_).

Também iremos nos atentar a outros fatores que podem influenciar um desfecho mais sério:
- o momento do dia (coluna *fase_dia*) pode afetar a atenção (via modulação da atenção dos motoristas por conta do ritmo circadiano), luminosidade e movimento na via;
- o tipo de pista (coluna *tipo_pista*), que pode determinar o tipo de manobra e o envolvimento de outros veículos; [SUPRIMIDO POR HORA]
- a morfologia da pista (coluna *tracado_via*), uma vez que a presença de curvas e cruzamentos podem exacerbar as dificuldades em climas adversos;
- a classificação da via (urbana ou rural; coluna *uso_solo*) pode influenciar a velocidade de resgate e atendimento, influenciando o desfecho de saúde dos envolvidos.

\\

---

\\

**Algumas inspiraçoes teóricas:**

Igelström E, Craig P, Lewsey J, et al. _"Causal inference and effect estimation using observational data"_. J Epidemiol Community Health 2022;76:960–966.

"Causal inference with observational data: A tutorial on propensity score analysis", doi.org/10.1016/j.leaqua.2023.101678

# Configurações e preparação

In [1]:
# INSTALLS

In [2]:
#pip install unidecode

In [1]:
pip install causalinference

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 1.0 MB/s eta 0:00:00


In [1]:
# IMPORTS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import json
import scipy
import seaborn as sns

# ACCESS TO DATA
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Agregar os dados de vários anos em um único DataFrame

In [6]:
anos = list(range(2019,2022,1))

In [7]:
acidentes = [] # todos acidentes registrados entre 2007-2021
for ano in anos:
  acidentes_ano = pd.read_csv(f'./drive/MyDrive/data/acidentes/datatran{ano}.csv', sep = ';', encoding = 'latin1', decimal = ',').copy()
  acidentes_ano['ano'] = ano

  acidentes.append(acidentes_ano)
  del acidentes_ano # limpar a memória

In [8]:
acidentes_global = pd.concat(acidentes).copy()
print(f'Há {len(acidentes_global)} acidentes registrados no período 2007-2021')

Há 161895 acidentes registrados no período 2007-2021


In [9]:
acidentes_global.columns

Index(['id', 'data_inversa', 'dia_semana', 'horario', 'uf', 'br', 'km',
       'municipio', 'causa_acidente', 'tipo_acidente',
       'classificacao_acidente', 'fase_dia', 'sentido_via',
       'condicao_metereologica', 'tipo_pista', 'tracado_via', 'uso_solo',
       'pessoas', 'mortos', 'feridos_leves', 'feridos_graves', 'ilesos',
       'ignorados', 'feridos', 'veiculos', 'latitude', 'longitude', 'regional',
       'delegacia', 'uop', 'ano'],
      dtype='object')

# Modificações previstas no conjunto de dados

1. Criar uma coluna "score de fatalidade" como delineado acima (vítimas fatais/número de envolvidos no acidente) ✔
2. Omitir as demais colunas (i.e. _drop_ em uma cópia do dataframe) ✔
3. Remover linhas com valor nulo para as variáveis de interesse (no futuro pode-se fazer uma inputação; agora não disponho de tanto tempo); ✔
4. Transformar os rótulos textuais da condição climática para um score: 1 = {céu claro, sol}, 2 = {nublado, vento, garoa/chuvisco}, 3 = {chuva, neve, nevoeiro/neblina, granizo}; ✔
5. Fazer transformações similares de rótulo textual para integer. Convenção: _valores menores serão imputados a condições menos adversas (e.g. via reta, pleno dia, via urbana, etc.) e valores mais altos a condições mais adversas (e.g. pista curva, plena noite, via rural, etc.). ✔

In [19]:
# 1. Criar uma coluna "score de fatalidade" como delineado acima (vítimas fatais/número de envolvidos no acidente) ✔
acidentes_global['score_fatalidade'] = acidentes_global['mortos']/acidentes_global['pessoas']
acidentes_global['score_ilesos'] = acidentes_global['ilesos']/acidentes_global['pessoas']

In [36]:
# 2. Omitir as demais colunas

target_vars = ['fase_dia','condicao_metereologica','tipo_pista','tracado_via','score_ilesos','score_fatalidade']

acidentes_df = acidentes_global[target_vars].copy()

In [38]:
# 3. Remover linhas com valor nulo para as variáveis de interesse (no futuro pode-se fazer uma inputação; agora não disponho de tanto tempo)

# O dataframe tem variação nesses rótulos por conta de unicode/capitalização. Vamos modificar isso tirando os acentos e capitalizando tudo
#for col in ['condicao_metereologica','fase_dia','tipo_pista','tracado_via','uso_solo']:
for col in ['condicao_metereologica','fase_dia','tipo_pista','tracado_via']:
  acidentes_df[col] = acidentes_df[col].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
  acidentes_df[col] = acidentes_df[col].str.upper()

#print(acidentes_df.condicao_metereologica.unique()) # A var. causal tem valores NaN e strings '(null)' e 'Ignorada' que não são informativos e vamos remover as linhas que os contenham
print('# antes da filtragem:',len(acidentes_df))

# antes da filtragem: 161895


In [39]:
# Remover linhas cujo valor na coluna-alvo esteja na lista de valores passada
def filter_entry_by_column_value(df, column, values):
  lines_with_condition = df[column].isin(values)
  return df[~lines_with_condition].copy()

# Variável de interesse (causal)
acidentes_df = filter_entry_by_column_value(acidentes_df,'condicao_metereologica',values=['(NULL)','IGNORADA','IGNORADO',np.nan])

# Outras variáveis
acidentes_df = filter_entry_by_column_value(acidentes_df,'tracado_via',values=['(NULL)','NAO INFORMADO',np.nan])
acidentes_df = filter_entry_by_column_value(acidentes_df,'tipo_pista',values=['(NULL)'])
acidentes_df = filter_entry_by_column_value(acidentes_df,'fase_dia',values=['(NULL)',np.nan])
#acidentes_df = filter_entry_by_column_value(acidentes_df,'uso_solo',values=['NAO','SIM']) # não sabemos o que são as tags "não"/"sim"

#print(acidentes_df.condicao_metereologica.unique())
print('# depois da filtragem:',len(acidentes_df))

# depois da filtragem: 141313


In [40]:
# 4. Transformar os rótulos textuais da condição climática para um score: 1 = {céu claro, sol}, 2 = {nublado, vento, garoa/chuvisco}, 3 = {chuva, neve, nevoeiro/neblina, granizo}

# Definir os novos rótulos

# Condições metereológicas:
condicoes_dict = {'CEU CLARO':0,'SOL':0,
                  'NUBLADO':1,'VENTO':1,'GAROA/CHUVISCO':1,
                  'CHUVA':2,'NEVOEIRO/NEBLINA':2,'GRANIZO':2,'NEVE':2}

# Uso do solo:
uso_solo_dict = {'RURAL': 0, 'URBANO':1}

# Fase do dia:
fase_dia_dict = {'PLENO DIA': 0,
                 'AMANHECER':1,'ANOITECER':1,
                 'PLENA NOITE':2}

# Tipo de pista:
tipo_pista_dict = {'SIMPLES': 0,'DUPLA': 0,'MULTIPLA': 0} # não me convenci imediatamente de que uma oferece mais periculosidade que a outra então por enquanto deixo todas com mesmo valor

# Traçado da via:
# Aplicar apenas à reta um valor menor (decisão arbitrária)
tracados_dict = {tracado:1 for tracado in acidentes_df.tracado_via.unique()}
tracados_dict.update({'RETA':0})

# Fazer as substituições
acidentes_df.replace(to_replace={'condicao_metereologica':condicoes_dict},inplace=True)
acidentes_df.replace(to_replace={'uso_solo':uso_solo_dict},inplace=True)
acidentes_df.replace(to_replace={'tracado_via':tracados_dict},inplace=True)
acidentes_df.replace(to_replace={'fase_dia':fase_dia_dict},inplace=True)
acidentes_df.replace(to_replace={'tipo_pista':tipo_pista_dict},inplace=True)

# Estudo ingênuo do efeito médio do tratamento ("Naïve" ATE)

A ocorrência de chuva, neblina ou outras intempéries climáticas é um evento aleatório, que independe das demais variáveis acerca do trajeto ou viagem, e.g. traçado da via. Uma análise trivial seria simplesmente comparar os acidentes que ocorreram sob bom e mau tempo.

Uma métrica simples para essa comparação é o ATE: average treatment effect, efeito médio do tratamento. Ele é simplesmente a diferença entre as médias de cada grupo, o grupo controle (que não teve intervenção, i.e. bom tempo) e o grupo tratamento (que teve intervenção, i.e. mau tempo).

Se pudéssemos observar pra cada observação $i$ tanto o valor da variável-alvo (nesse caso, porcentagem de ilesos no acidente) sob bom tempo $y_i(mau \; tempo = 0)$ quanto sob mau tempo $y_i(mau \; tempo = 1)$ bastaria que fizéssemos a média das diferenças em todas as observações:

\begin{equation}
ATE = \frac{1}{N} \sum_i \big ( y_i(mau \; tempo = 1) - y_i(mau \; tempo = 0) \big )
\end{equation}

Como não dispomos do desfecho simultâneo para uma observação com e sem tratamento esse $ATE$ pode ser calculado como a média de todas observações em cada condição:

\begin{equation}
\widehat{ATE} = \frac{1}{N_i} \sum_i y_i(mau \; tempo = 1) - \frac{1}{N_j} \sum_j y_j(mau \; tempo = 0) \\
\quad \quad = \mathbb{E}[y_i(mau \; tempo = 1)] - \mathbb{E}[y_j(mau \; tempo = 0)]
\end{equation}

onde $i$ são as ocorrências no grupo com intervenção (direção sob mau tempo) e $j$ são as ocorrências no grupo sem (bom tempo).

In [41]:
grupo_controle = acidentes_df.query('condicao_metereologica < 1').copy()
grupo_intervencao = acidentes_df.query('condicao_metereologica >= 1').copy()

print('Observações no grupo controle: ',len(grupo_controle),'\nObservações no grupo intervenção: ',len(grupo_intervencao))

Observações no grupo controle:  95943 
Observações no grupo intervenção:  45370


Como ambos conjuntos são grandes ($>100.000$ observações), aplica-se a lei dos grandes números e é justificável uma média.

A diferença no número de observações em cada grupo é bem grande, ~1 mi observações. Em etapas futuras pode-se fazer um controle com reamostragem (e.g. bootstrapping) da amostra maior.

Sem mais delongas, calculemos o ATE.

In [42]:
media_controle = grupo_controle.score_ilesos.mean()
media_intervencao = grupo_intervencao.score_ilesos.mean()
print(media_controle,media_intervencao)

0.38385461500540713 0.3771807193250202


In [43]:
ate = media_intervencao - media_controle
print('ATE ',ate)

ATE  -0.006673895680386954


# Pareamento de score de propensidade (PSM) e efeito médio do tratamento nos tratados (ATT)

Contudo, nosso conjunto de dados pode estar enviesado. Diferentemente de ensaios clínicos aleatorizados, temos apenas observações de eventos. Pode ser que hajam, por exemplo, mais vítimas em acidentes na chuva porque eles mais frequentemente acontecerem em pontes ou curvas.

\

Para garantir que o efeito observado seja atribuível exclusivamente à intempérie metereológica podemos nos inspirar na técnica de pareamento de score de propensidade (tradução livre de propensity score matching, PSM). Formam-se pares entre itens no grupo de intervenção e controle que sejam similares em todas demais variáveis confusoras (i.e. outros fatores que influenciam o desfecho, e.g. traçado da via no nosso caso) exceto na variável cujo efeito causal no desfecho queremos avaliar.

\

Ela funciona da seguinte maneira:


1. Atribui a cada observação um identificador com relação às variáveis confusoras (e.g. com um score calculado a partir de uma regressão logística)
2. Encontra pares P_i = {intervenção, controle} de observações em cada condição que tenham identificadores parecidos (e.g. scores similares) utilizando alguma técnica (e.g. vizinhos próximos ou distância num espaço N-dimensional) e armazena cada elemento do par em uma distribuição correspondente à condição (e.g. a observação do grupo de intervenção vai pra uma lista de observações do grupo controle já pareadas; similarmente pra observação do grupo controle)
3. Pareados todos elementos no grupo intervenção, verifica-se se a distribuição de cada variável confusora é similar em ambas listas (e.g. através de um teste t comparando as médias)
4. Se as covariâncias nos dois conjuntos for similar, a ATE calculada usando as observações em cada conjunto vai ser atribuível tão somente à variável causal


In [34]:
from scipy.spatial import KDTree

In [44]:
# Transforma as observações no grupo de intervenção (mau tempo) em uma estrutura de dados KDTree
desfecho_intervencao = grupo_intervencao['score_ilesos']

In [57]:
# cria a estrutura de dados árvore k-d com dados do grupo controle, permitindo uma varredura eficiente por proximidade com base nas dimensões de interesse (variáveis confusoras)
kd_controle = KDTree(grupo_controle.drop(columns=['score_ilesos','score_fatalidade','condicao_metereologica']).values)

In [59]:
# buscar na árvore k-d gerada com os dados do grupo controle o id da observação mais próxima de cada observação no grupo intervenção (efetivamente: pares {controle, intervenção})
print(kd_controle.query(grupo_intervencao.drop(columns=['score_ilesos','score_fatalidade','condicao_metereologica']).values, k=1)[-1])

[82410 51958 51958 ... 79984 82410 79984]


In [64]:
# dataframe com os dados desse grupo controle
controle_pareado = grupo_controle.iloc[kd_controle.query(grupo_intervencao.drop(columns=['score_ilesos','score_fatalidade','condicao_metereologica']).values, k=1)[-1]]
controle_pareado

,fase_dia,condicao_metereologica,tipo_pista,tracado_via,score_ilesos,score_fatalidade
8483,2,0,0,0,0.0,0.0
21647,2,0,0,1,1.0,0.0
21647,2,0,0,1,1.0,0.0
8483,2,0,0,0,0.0,0.0
21647,2,0,0,1,1.0,0.0
...,...,...,...,...,...,...
21647,2,0,0,1,1.0,0.0
4094,0,0,0,0,0.5,0.0
4094,0,0,0,0,0.5,0.0
8483,2,0,0,0,0.0,0.0


In [ ]:
# teste das variáveis confusoras entre o grupo controle pareado e o grupo de interesse


In [51]:
len(kd_intervencao.indices)

45370

In [16]:
Y = acidentes_df['score_fatalidade'].values
#X = acidentes_df['condicao_metereologica' >= 1].values
X = (acidentes_df['condicao_metereologica'] >= 1).values

confounders = acidentes_df.copy()
print(confounders.tracado_via.unique())
confounders.drop(columns=['score_fatalidade','condicao_metereologica'],inplace=True)
print(confounders.tracado_via.unique())

[0 1]
[0 1]


In [17]:
confounders.columns

Index(['fase_dia', 'tipo_pista', 'tracado_via', 'uso_solo'], dtype='object')

In [21]:
confounders.uso_solo.unique()

array([0, 1])

In [23]:
from causalinference import CausalModel
causal = CausalModel(Y, X, confounders)

In [24]:
causal.est_via_matching()

UFuncTypeError: ufunc 'subtract' did not contain a loop with signature matching types (dtype('int64'), dtype('<U8')) -> None

# Rubin causal model

* Stable unit treatment value assumption (SUTVA) ✔
* ATE
* constant effect




\begin{equation}
fatality_i = \beta_0 + \beta_1 * \delta_i + u_i
\end{equation}

$\beta_0 := $

$\beta_1 := $ efeito do mau tempo

$\delta_i := $ se o acidente $i$ aconteceu sob mau tempo

$u_i := $ efeito aleatório (erro)

o mau tempo é um evento aleatório, mas nós podemos ter vieses na amostra: e se com a previsão de mau tempo as pessoas não saírem de casa tanto quanto em ocasiões de bom tempo, ou dirigirem com mais cautela? precisamos comparar cenários similares em ambas variáveis em termos dos vários fatores observados.

e.g. digamos que hajam mais vítimas em acidentes sob mau tempo que acontecem em pistas curvas.

In [12]:
acidentes_df

,fase_dia,condicao_metereologica,tipo_pista,tracado_via,uso_solo,score_fatalidade
0,1,1,DUPLA,1,0,0.0
1,1,3,DUPLA,1,0,0.0
2,1,1,SIMPLES,1,0,0.0
3,3,1,SIMPLES,1,1,0.4
4,2,1,DUPLA,1,0,0.0
...,...,...,...,...,...,...
96358,1,1,SIMPLES,1,1,0.5
96359,3,1,SIMPLES,1,1,0.0
96360,1,3,SIMPLES,2,0,0.0
96361,1,1,SIMPLES,1,1,0.0


In [ ]:
est_via_matching(self, weights='inv', matches=1, bias_adj=False)

# logica da analise

* chuva é um evento aleatório, podemos só calcular ATT e ver o impacto
* contudo esse conjunto pode sim ter vieses porque a gente tá olhando pra ocorrências que aconteceram e foram registradas; pode ser que tenhamos mais acidentes numa interação condição metereológica desfavorável-tipo de pista-tracado da via do que em uma condição meteorológica favorável
* isso justifica uma abordagem de propensity score matching